# Keck HIRES Precision Radial Velocity Data Reduction 

<b><i>This service enables reduction and analysis of precision radial velocity (PRV) data from the HIRES Keck instrument. </i></b>

<i><b><font style="color: red;">This notebook is not part of the official release, not even for alpha testing.  The project found it useful for its own testing but makes no guarantees it will even work for anyone else.  Use, and even read, at your own risk.</font></b></i>

This notebook introduces the Keck HIRES Precision Radial Velocity (PRV) pipeline service and works through one specific example. There are number of variations, mostly having to do with planning processing, which will be covered in more detail by other notebooks but here you will see all the basics.

The notebook is kept with the HIRES PRV Python access toolkit in <a href="https://github.com/Caltech-IPAC/hiresprv">GitHub: https://github.com/Caltech-IPAC/hiresprv</a>

## Login

Logging in the first time creates a workspace for the user and associates it with a KOA account.

Users of this service must have Keck On-line Archive (KOA) accounts and use that login here to gain access to their data.  Even researchers planning to use only public data will need a KOA login as this service is maintaining persistent storage under that ID.

The login is persisted through the use of HTTP cookies and logging in from multiple clients will connect the user to the same account, storage, and processing history.  This environment (user workspace) is permanent as we expect some on-going research to span years.  The login for a give client machine need only be done once, assuming the cookie file in the local storage is not deleted.  If it is, logging in again will reconstruct it.

The cookie file, processing state information, and downloaded results like 1D spectra and RV curve tables will be kept locally in the same space as this notebook.  If you wish to change that, simply add in whatever directory management and navigation you like.

In [ ]:
from IPython.display import IFrame
from hiresprv.auth import login

login('prv.cookies')


## KOA Data Retrieval

The PRV workspace first needs to be populated with data from the KOA Archive.  This can be done all at once, if the data exists, or incrementally as the data is taken/identified.

This step is more than a simple data transfer.  "Raw reduction" of the data, which converts the 2D CCD spectal images to 1D spectra, is done up-front as the data is retrieved a night at a time.  The UT dates you give here are actually shifted a few hours to catch any calibration data collected in the afternoon of the same (Hawaii-local) day.

In [ ]:
from hiresprv.archive import Archive

koa = Archive('prv.cookies')

rtn = koa.by_dates("""2009-12-31
2013-06-29
2013-09-12
2014-06-06""")




Note that since the data in the workspace is permanent, repeated request for the same data would not change anything and so those dates will be ignored.  Therefore, you can add to the above list or replace it with new dates as you choose.  Dates must be formatted YYYY-MM-DD (a KOA requirement).

The above service responds immediately with an acknowledgement of the request and starts the actual transfer and raw data reduction (which can take some time) as a background job.  The job status can be checked by polling or can be monitored using the function below.  While one retrieval job (or processing job below) is running, no others can be initiated.

## PRV Processing Monitor

The PRV monitor makes use of IFrames.  An IFrame is a region in the Jupyter page which can display another full browser document, in this case a real-time web service watching the workspace status file.  Unlike the rest of the Jupyter page, where we have very little control over layout and event management, the IFrame contents can be treated much more like a traditional HTML/Javascript GUI.

In [ ]:
from hiresprv.status import Status

monitor = Status('prv.cookies')

url = monitor.processing_status()

IFrame(url, 1000, 500)

## Metadata

Once data has been retrieve and the "nightly" raw reduction performed, a set of records is added to a persistent metadata table, one per observation.  These observations are all taken through the HIRES PRV instrument (2D CCD) and will have been reduced to 1D spectra by the raw reduction.  They fall into five classes:

<ul>
<li><b>RV observations</b> -- Multiple observations of a star with the iodine cell in the light path.  These are the measurements that each actually result in a calculation of a radial velocity.<p/></li>
    
<li><b>Templates</b> -- One long observation of the same star without iodine, for reference.<p/></li>
    
<li><b>B stars</b> -- A set of observations of rapidly rotating B stars bracketing the template observation and used to reduce it.<p/></li>
    
<li><b>Iodine</b> -- Reference observation of iodine for nightly calibration.<p/></li>
    
<li>Miscellaneous other calibration observations (labelled as "<b>Unknown</b>").<p/></li>
</ul>

By inspecting this table, the user can determine what objects were observed, whether there are template observation for which of them (and adequate B star data to reduce a template), and whether there are enough RV measurements to generate a final RV curve.

With a small metadata table, this is simple enough to do by inspection but a typical workspace can easily have thousands of files covering tens or hundreds of objects.  Furthermore, since observations for a single object are frequently spread out over years, the metadata table is often fairly thorougly mixed in time.

Luckily, there are a number of tools available in client-side Python subset and organize the metadata, so we provide it for download as a CSV table or an SQLite binary file or even, as here, as a simple HTML table.  The workspace copy of the data is maintained in an SQLite database so we also provide a basic filtering mechanism as an optional addition to the download.  This filtering is often adequate for basic processing scenarios.  

Note that metadata retrieval can't be done while the system is "busy" (downloading additional data or further reducing data in the workspace). Otherwise, metadata downloads can be done at any time.

Also note that the client-side file will become out-of-date once new data download or processing requests are submitted.  It is up to the user to re-request the new metadata.


In [ ]:
from hiresprv.database import Database

state = Database('prv.cookies')

url = state.search()

IFrame(url, 1000,  500)

## Subsetting the Metadata: Single Target

Ultimately, to make an RV curve for one star we need to reduce its observations into RV measurements.  Assuming there are adequate B star observations to reduce the template, we can isolate appropriate records in the metadata above by simply filtering on TARGET name.  There are many ways to do this; in our case we we used the remote SQLite query capability and filtered it with <p/>

<tt>select DATE, OBTYPE, FILENAME, TARGET, BJD, BCVEL from FILES where TARGET like 'HD185144';</tt> 

The resulting data is shown below.

In [ ]:
url = state.search(sql="select DATE, OBTYPE, FILENAME, TARGET, BJD, BCVEL from FILES where TARGET like 'HD185144';")

IFrame(url, 700,  325)

### RV Pipeline Processing

This shows that on 12/31/2009 three separate RV observations were made of HD185144 followed by five template observations (which the pipeline will combine into a single template).  Five years later, another three RV observations were made.

As with the data download, the further reduction steps in the pipeline can be quite lengthy (minutes to hours each), so rather than have the user monitor each one, we provide a scripting mechanism so complex reduction jobs can be submitted in on shot.

In order to turn any of the RV observations into an RV value, we need the template.  So we will generate that first.  Since it is possible to repeat the template observations on more than one day, we need to explicitly state which object and which day.  The script command for this is:

<pre>template HD185144 20091231</pre>

To reduce an RV measurement, we have to refer to this template (the target name is enough) and specify which file to reduce.  For example:

<pre>rv HD185144 r20091231.7</pre>

Finally, once we have a set of RV measurements for an object, we a generate an RV curve (the pipeline finds all the appropriate RV measurements):

<pre>rvcurve HD185144</pre>

As long as we follow the general rules that we need a template before we can reduce an RV measurement and we need at least three RV measurements before we can generate an RV curve, we can otherwise scripte things in whatever order we wish (<i>e.g.</i> all the templates first).

All of this is submitted to the pipeline as a text script:



In [ ]:
from hiresprv.idldriver import Idldriver

idl = Idldriver('prv.cookies')

rtn = idl.run_script("""
template HD185144 20091231
rv HD185144 r2R0091231.72
rv HD185144 r20091231.73
rv HD185144 r20091231.74
rv HD185144 r20150606.145
rv HD185144 r20150606.146
rv HD185144 r20150606.147
rvcurve HD185144
""")

print(rtn)

### Monitoring (again)

To monitor the pipeline processing request, you can either insert another cell the runs the PRV Processing Monitor here or just go back to the previous cell and monitor from there.


### Data Retrieval

The principal retrievable items are the RV curves (CSV files) for each target

In [ ]:
#  rvdata = prvDataRetrieval(target='HD185144')
#  print(rvdata)

url='HD185144.txt'
IFrame(url, 700,  325)